## Overview

"Mnemo" classes wrap Sequence & Mapping objects and log object interaction.

Each Mnemo class inherits from the corresponding Abstract Base Class (ABC), and
thus supports all methods that are available by default to Sequence or Mapping 
objects. 

The purpose of the Mnemo wrapper is to log actions taken on the object (both 
with and without side-effects). The set of actions captured are 
`(__getitem__, __setitem__, __delitem__, insert)`. Since these are
abstract methods, calling a method like append (that implements the abstract
methods) will result in `__getitem__` and `__setitem__` being logged multiple
times.

The log is stored in self.log as a list of tuples: 

    [(ctr, action_type, index/key, value)...]

... where: 

    ctr = int of action counter, starts from 0 and autoincrements by 1
    action_type = string of action type, one of ('get', 'set', 'del', 'ins')
    index/key = index or key being used
    value = value being retrieved or effected


## Classes
* MnemoSequence: class for non-mutable sequences, e.g. strings, tuples
* MnemoMutableSequence: class for mutable sequences, e.g. lists
* MnemoMutableMapping: class for mutable mappings, e.g. dicts

In [1]:
import collections

In [2]:
class MnemoSequence(collections.Sequence):
    """Mnemo class for Sequence objects. 
    Args
        obj: Sequence object with which to initialize object, e.g. tuple
    Methods
        abstract methods required by ABC: __getitem__, __len__    
    """

    def __init__(self, obj):
        self.obj = obj
        self.ctr = 0
        self.log = []

    def __getitem__(self, ind):
        self.log.append((self.ctr, 'get', ind, self.obj[ind]))
        self.ctr += 1
        return self.obj[ind]

    def __len__(self):
        return len(self.obj)

In [3]:
class MnemoMutableSequence(collections.MutableSequence):
    """Mnemo class for Mutable Sequence objects.
    Args
        obj: Mutable Sequence object with which to initialize object, e.g. list
    Methods
        abstract methods required by ABC: __getitem__, __setitem__, 
                                          __delitem__, __len__, insert
    """
    def __init__(self, obj):
        self.obj = obj
        self.ctr = 0
        self.log = []

    def __getitem__(self, ind):
        self.log.append((self.ctr, 'get', ind, self.obj[ind]))
        self.ctr += 1
        return self.obj[ind]

    def __setitem__(self, ind, val):
        self.log.append((self.ctr, 'set', ind, val))
        self.ctr += 1
        self.obj[ind] = val

    def __delitem__(self, ind):
        self.log.append((self.ctr, 'del', ind, self.obj[ind]))
        self.ctr += 1
        del self.obj[ind]

    def __len__(self):
        return len(self.obj)

    def insert(self, ind, val):
        self.log.append((self.ctr, 'ins', ind, val))
        self.ctr += 1
        self.obj.insert(ind, val)


In [4]:
class MnemoMutableMapping(collections.MutableMapping):
    """Mnemo class for Mutable Mapping objects.
    Args
        obj: Mutable Mapping object with which to initialize object, e.g. dict
    Methods
        abstract methods required by ABC: __getitem__, __setitem__, 
                                          __delitem__, __iter__, __len__
        insert: insert new key (and value) into object
        check_valid: helper function for abstract methods
    """

    def __init__(self, obj):
        self.obj = obj
        self.ctr = 0
        self.log = []

    def __getitem__(self, key):
        self.check_valid(key)
        self.log.append((self.ctr, 'get', key, self.obj[key]))
        self.ctr += 1
        return self.obj[key]

    def __setitem__(self, key, val):
        self.check_valid(key, val)
        self.log.append((self.ctr, 'set', key, val))
        self.ctr += 1
        self.obj[key] = val

    def __delitem__(self, key):
        self.check_valid(key)
        self.log.append((self.ctr, 'del', key, self.obj[key]))
        self.ctr += 1
        del self.obj[key]

    def __iter__(self):
        return iter(self.obj.keys())

    def __len__(self):
        return len(self.obj.keys())

    def insert(self, key, val):
        self.log.append((self.ctr, 'ins', key, val))
        self.ctr += 1
        self.obj[key] = val
    
    def check_valid(self, key, val=''):
        """Check if key is in object's keys, else insert."""
        if key not in self.obj.keys():
            if val:
                self.insert(key, val)
            else:
                raise KeyError, key

<hr>

## Examples

### List

In [5]:
sample_list = list('abcdefg')
test = MnemoMutableSequence(sample_list)
test.obj

['a', 'b', 'c', 'd', 'e', 'f', 'g']

In [6]:
test.log

[]

In [7]:
test[0]

'a'

In [8]:
test[1] == 'Z'

False

In [9]:
test[1] = 'Z'

In [10]:
del test[-1]

In [11]:
test.obj

['a', 'Z', 'c', 'd', 'e', 'f']

In [12]:
test.log

[(0, 'get', 0, 'a'),
 (1, 'get', 1, 'b'),
 (2, 'set', 1, 'Z'),
 (3, 'del', -1, 'g')]

### Dictionary

In [13]:
sample_d = {'a': 123,
            'b': 456,
            'c': 789}
test_d = MnemoMutableMapping(sample_d)

In [14]:
test_d['a']

123

In [15]:
test_d['d'] = 999

In [16]:
test_d.obj

{'a': 123, 'b': 456, 'c': 789, 'd': 999}

In [17]:
test_d.log

[(0, 'get', 'a', 123), (1, 'ins', 'd', 999), (2, 'set', 'd', 999)]

<hr>

## Tests

In [18]:
def test_MnemoSequence():
    """Unit tests for MnemoSequence using str, unicode, and tuple examples."""

    # Create and find length of empty sequence
    for empty in ('', u'', ()):
        test = MnemoSequence(empty)
        assert len(test) == 0, 'Empty sequence should have length 0'

    # Create and find length of singleton sequence
    for singleton in ('a', u'a', ('a',)):
        test = MnemoSequence(singleton)
        assert len(test) == 1, 'Singleton sequence should have length 1'

    # Access non-empty sequence and verify log is written
    for seq in ('abc', u'abc', ('a', 'b', 'c')):
        test = MnemoSequence(seq)
        test[0]
        assert test.log == [(0, 'get', 0, 'a')], 'Log incorrect'

    return True

In [20]:
def test_MnemoMutableSequence():
    """Unit test for MnemoMutableSequence using list as an example."""

    # Create and find length of empty sequence
    for empty in ([],):
        test = MnemoMutableSequence(empty)
        assert len(test) == 0, 'Empty sequence should have length 0'

    # Create and find length of singleton sequence
    for singleton in ([1], ['a']):
        test = MnemoMutableSequence(singleton)
        assert len(test) == 1, 'Singleton sequence should have length 1'

    # Access non-empty sequence and verify log
    for seq in (['a', 'b', 'c'], ['a', 1, 2, 3]):
        test = MnemoMutableSequence(seq)
        test[0]
        assert test.log == [(0, 'get', 0, 'a')], 'Log incorrect (get)'

    # Set item and verify log
    for seq in (['!', '-', '*'], ['&', '%', '*']):
        test = MnemoMutableSequence(seq)
        test[2] = '$'
        assert test.log == [(0, 'set', 2, '$')], 'Log incorrect (set)'

    # Delete item and verify log
    for seq in (['!', '-', '*'], ['&', '%', '*']):
        test = MnemoMutableSequence(seq)
        del test[2]
        assert test.log == [(0, 'del', 2, '*')], 'Log incorrect (del)'

    # Insert item and verify list, length, and log
    for seq in ([1, 2, 3, 4, 5],):
        test = MnemoMutableSequence(seq)
        test.insert(2, 100)
        assert test.obj == [1, 2, 100, 3, 4, 5], 'Insertion object incorrect'
        assert len(test) == 6, 'Insertion object results in incorrect length'

    # Test reverse(); should result in many log entries
    for seq in ([1, 2, 3, 4, 5],):
        test = MnemoMutableSequence(seq)
        test.reverse() 
        assert test.obj == [5, 4, 3, 2, 1], 'Reversed object incorrect'
        assert len(test.log) > 1, 'Log length too short after reverse'

    return True

In [19]:
def test_MnemoMutableMapping():
    """Unit tests for MnemoMutableMapping using dict as example."""

    # Create and find length of empty mapping
    for empty in ({},):
        test = MnemoMutableMapping(empty)
        assert len(test) == 0, 'Empty sequence should have length 0'

    # Create and find length of singleton sequence
    for singleton in ({1: 'a'}, {2: 'b'}):
        test = MnemoMutableMapping(singleton)
        assert len(test) == 1, 'Singleton sequence should have length 1'

    # Access non-empty sequence and verify log
    for seq in ({0: 'a', 1: 'b'}, {0: 'a', 100: 'b'}):
        test = MnemoMutableMapping(seq)
        test[0]
        assert test.log == [(0, 'get', 0, 'a')], 'Log incorrect (get)'

    # Set item and verify log
    for seq in ({0: 'a', 'testing': '#'}, {'a': 'b', 'testing': 100}):
        test = MnemoMutableMapping(seq)
        test['testing'] = '$'
        assert test.log == [(0, 'set', 'testing', '$')], 'Log incorrect (set)'

    # Delete item and verify log
    for seq in ({1: '#', 2: '*', 100: '?'}, ):
        test = MnemoMutableMapping(seq)
        del test[2]
        assert test.log == [(0, 'del', 2, '*')], 'Log incorrect (del)'

    # Check iterkeys works as expected
    for seq in ({0: 'zero', 1: 'one', 1000: 'thousand'},):
        test = MnemoMutableMapping(seq)
        keys = test.iterkeys()
        assert sorted(list(keys)) == [0, 1, 1000], 'Iterkeys is wrong'

    return True

In [21]:
def test():
    """Execute unit tests."""
    print '\nMnemoSequence\n', test_MnemoSequence()
    print '\nMnemoMutableSequence\n', test_MnemoMutableSequence()
    print '\nMnemoMutableMapping\n', test_MnemoMutableMapping()

if __name__ == '__main__':
    test()


MnemoSequence
True

MnemoMutableSequence
True

MnemoMutableMapping
True
